In [27]:
import pandas                as pd
import plotly                as py
import plotly.graph_objs     as go
import xlrd
import plotly.express        as px
import plotly.figure_factory as ff
from   datetime              import datetime, date
from   datetime              import timedelta
from   pptx                  import Presentation, util
from   pd2ppt                import df_to_table
from   pptx.util             import Inches

In [28]:
# Initial Addresses Information 
excel_address = '/Users/annademidova/PycharmProjects/HAR_Parser/Code/story_tables_generated.xlsx'
prs_path = '/Users/annademidova/PycharmProjects/HAR_Parser/SAP_Template_Presentation.pptx'
prs = Presentation(prs_path)
images_path = '/Users/annademidova/PycharmProjects/HAR_Parser/images/'

In [29]:
# Header Slide
slide = prs.slides[0]
slide.placeholders[0].text = 'Analysis of HAR Trace'
slide.placeholders[1].text = 'Anna Demidova'
prs.save(prs_path)

In [30]:
# Creating dataframes for total/average/maximum times
# Data Preparation
data = pd.read_excel(excel_address, sheet_name = 'story_metadata', index_col=[0]) 

# Creating Dataframes
df = pd.DataFrame(data)
wait_index = df.index.isin(['wait'])
values_total_runtime_full = df['total_runtime'].map('{:,.2f}'.format)
values_total_runtime = values_total_runtime_full[~wait_index]
values_average_runtime_full = df['average_runtime'].map('{:,.2f}'.format)
values_average_runtime = values_average_runtime_full[~wait_index]
values_minimum_runtime = df['minimum_runtime'].map('{:,.2f}'.format)
values_maximum_runtime = df['maximum_runtime'].map('{:,.2f}'.format)
labels = df.index

# Pie Charts creation
fig = px.pie(df, values=values_total_runtime, names=df[~wait_index].index, title='Total Runtime without Wait Time')
fig.write_image(images_path + 'TotalTime.png')

fig = px.pie(df, values=values_average_runtime, names=df[~wait_index].index, title='Average Runtime')
fig.write_image(images_path + 'AverageTime.png')

#fig = px.pie(df, values=values_minimum_runtime, names=df.index, title='Minimum Runtime')
#fig.write_image(images_path + 'MinimumTime.png')

fig = px.pie(df, values=values_maximum_runtime, names=df.index, title='Maximum Runtime')
fig.write_image(images_path + 'MaximumTime.png')

In [31]:
# Gantt chart 
# Data Praparation 
data_gantt = pd.read_excel(excel_address, sheet_name = 'story_runtime') 

# Creating DataFrames
df_gantt = pd.DataFrame(data_gantt)
df = []
for index, row in df_gantt.iterrows():
    df_dict = {}
    df_dict['Task'] = row['URL']
    df_dict['Start'] = datetime.strptime(row['Start Timestamp'], '%m.%d.%Y %H:%M:%S')
    df_dict['Finish'] = datetime.strptime(row['Start Timestamp'], '%m.%d.%Y %H:%M:%S') + timedelta(seconds=row['Total Time']/1000)
    df.append(df_dict)

fig = ff.create_gantt(df)
fig.write_image(images_path + 'GanttChart.png')

In [32]:
# Inserting Total/Average/Maximum Times Slides
slide = prs.slides[4]
slide.placeholders[0].text = 'Time Graphs'
slide.placeholders[17].text = 'Total and average time graphs are made with wait time excluded'
slide.placeholders[11].insert_picture(images_path + 'TotalTime.png')
slide.placeholders[15].text = 'Total Time without Wait Time'
slide.placeholders[12].insert_picture(images_path + 'AverageTime.png')
slide.placeholders[16].text = 'Average Times without Wait Time'
slide.placeholders[13].insert_picture(images_path + 'MaximumTime.png')
slide.placeholders[14].text = 'The biggest times'

prs.save(prs_path)

In [33]:
# Inserting Gantt Chart 
slide = prs.slides[5]
slide.placeholders[0].text = 'Gantt Chart over Total Trace Time'
picture = slide.placeholders[11].insert_picture(images_path + 'GanttChart.png')

# Memorize initial positions of a picture placeholder
pos_left, pos_top = picture.left, picture.top

available_width = picture.width
available_height = picture.height
image_width, image_height = picture.image.size
placeholder_aspect_ratio = float(available_width) / float(available_height)
image_aspect_ratio = float(image_width) / float(image_height)

picture.crop_top = 0
picture.crop_left = 0
picture.crop_bottom = 0
picture.crop_right = 0

# if the placeholder is "wider" in aspect, shrink the picture width while
# maintaining the image aspect ratio
if placeholder_aspect_ratio > image_aspect_ratio:
    picture.width = int(image_aspect_ratio * available_height)
    picture.height = available_height
    picture.left, picture.top = pos_left, pos_top
    
# otherwise shrink the height
else:
    picture.height = int(available_width/image_aspect_ratio)
    picture.width = available_width
    picture.left, picture.top = pos_left, pos_top

prs.save(prs_path)

In [34]:
# Total Data Information
# Data Preparation
data = pd.read_excel(excel_address, sheet_name = 'general_info', index_col=[0], skiprows=1) 
df = pd.DataFrame(data)

# Adding slide to a Presentation
slide = prs.slides[6]
slide.placeholders[0].text = 'Total Story Info'
top = Inches(1.5)
left = Inches(1.0)
width = Inches(9.25)
height = Inches(5.0)
df_to_table(slide, df.reset_index(), left, top, width, height)
prs.save(prs_path)

In [35]:
# Get the list of sheets' names
all_sheets = pd.read_excel(excel_address, None)
# display(all_sheets['Get Response Timings 1'][['Description', 'Time']].dropna())
name_template = 'Get Response Timings '
for key, value in all_sheets.items():
     if name_template in str(key):
            # Get "Timings" Sheet
            df = all_sheets[key][['Description', 'Time']].dropna()
            # Create dataframe for a pie chart + pie chart
            values = df['Time'].map('{:,.2f}'.format)
            labels = df['Description']
            fig = px.pie(df, values = values, names = labels, title = key + ' (Data Fetching Times)')
            fetch_picture_name = key + ' (Data Fetching Times)'
            fig.write_image(images_path + fetch_picture_name + '.png')
            
            # Find corresponding "Meta" Sheet
            key_meta = str(key).replace('Timings', 'Meta')
            df_meta = all_sheets[key_meta]
            
            # If there is Widget Info inside - create additional table or create empty DataFrame
            if  'Widget ID' in df_meta.columns:
                df_widget_info = df_meta[['Story Name', 'Page Title', 'Widget ID',
                'Widget Class', 'Widget Title', 'Widget Name', 'Widget TTFB']].T
                df_widget_info = df_widget_info.reset_index()
                df_widget_info.columns = ['Info','Values']
            else:
                df_widget_info = pd.DataFrame()
                
                
            # Pie chart for network times creation
            df_net_times = df_meta[['blocked', 'dns', 'ssl', 'connect', 'send',
            'wait', 'receive', '_blocked_queueing']].T
            values = df_net_times[0].map('{:,.2f}'.format)
            labels = df_net_times.index
            fig = px.pie(df_net_times, values = values, names = labels, title = str(key).replace('Meta', 'Timings') + ' (Network Times)')
            network_picture_name = key_meta + ' (Network Times)'
            fig.write_image(images_path + network_picture_name + '.png')
            
            # Table for Link Information
            df_link_info = df_meta[['Start Timestamp', 'Total Time', 'Runtime', 'Body Size',
            'Status', 'Transfer Size']].T
            df_link_info = df_link_info.reset_index()
            df_link_info.columns = ['Info','Values']
            
            # Insert slide with times pictures
            pictures_slide = prs.slides.add_slide(prs.slide_layouts[7])
            pictures_slide.placeholders[0].text = key
            pictures_slide.placeholders[13].insert_picture(images_path + fetch_picture_name + '.png')
            pictures_slide.placeholders[15].insert_picture(images_path + network_picture_name + '.png')
            prs.save(prs_path)
            
            # Insert slide with tables
            tables_slide = prs.slides.add_slide(prs.slide_layouts[17])
            tables_slide.placeholders[0].text = key_meta
            # Insert link info tables
            if len(df_link_info.index) <= 5:
                top = Inches(1.5)
                left = Inches(1.0)
                width = Inches(4.25)
                height = Inches(2.0)
                df_to_table(tables_slide, df_link_info, left, top, width, height)
                prs.save(prs_path)
            elif len(df_link_info.index) > 5:
                top = Inches(1.5)
                left = Inches(1.0)
                width = Inches(5.0)
                height = Inches(4.0)
                df_to_table(tables_slide, df_link_info, left, top, width, height)
                prs.save(prs_path)
            #Insert widget table if exists
            if not df_widget_info.empty:
                top = Inches(1.5)
                left = Inches(7.0)
                width = Inches(5.0)
                height = Inches(4.0)
                df_to_table(tables_slide, df_widget_info, left, top, width, height)
                prs.save(prs_path)
                

In [179]:
slide = prs.slides[7]
print(prs.slide_layouts[17])
for shape in slide.placeholders:
    print('%d %s %s' % (shape.placeholder_format.idx, shape.name, shape.placeholder_format.type))

0 Заголовок 3 TITLE (1)
13 Рисунок 1 PICTURE (18)
15 Рисунок 2 PICTURE (18)
